<a href="https://www.goes-r.gov/">
<img src='./GOES-R_logo_small.png' style="height:150px" align=right alt="GOES16 Logo">
</a>

# GOES-16: Fire Temperature
__Brian Blaylock__   
__July 5, 2018__  
| brian.blaylock@utah.edu | [website](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/home.html) |  

Supplimental Notebooks | [GOES16 TrueColor](https://github.com/blaylockbk/pyBKB_v3/blob/master/BB_GOES/mapping_GOES16_TrueColor.ipynb)

This Python 3 notebook shows how to make the fire temperature product from the GOES-16 Advanced Baseline Imager (ABI) level 2 data. We will plot the image with `matplotlib` and `Basemap`. The image can be displayed on any map projection after applying a transformation using `pyproj`. The methods shown here are stitched together from the following useful information found online:
- [ABI Bands Quick Information Guides](https://www.goes-r.gov/education/ABI-bands-quick-info.html)
- [Open Commons Consortium](http://edc.occ-data.org/goes16/python/)
- [GeoNetCast Blog](https://geonetcast.wordpress.com/2017/07/25/geonetclass-manipulating-goes-16-data-with-python-part-vi/)
- [Proj documentation](https://proj4.org/operations/projections/geos.html?highlight=geostationary)
- [Pyproj documentation](http://jswhit.github.io/pyproj/pyproj.Proj-class.html)
- [Fire Temperature Recipe](OR_ABI-L2-MCMIPC-M3_G16_s20182570022128_e20182570024501_c20182570025006.nc)

The **Fire Temperature Product** is an RGB composite of the following three channels:


|         _ | Wavelength   | Channel | Description |
|----------|:------------:|:-------:|:-----------:|
| **Red**  | 3.9 &#181;m  |    7    | Shortwave Window |
| **Green**| 2.2 &#181;m |    6    | Cloud Particle Size|
| **Blue** | 1.6 &#181;m |    5    | Snow/Ice|

---

## Download a GOES-16 ABI file
For this demo, you will need GOES-16 ABI level 2 data. You can get GOES-16 files from NOAA's GOES archive on [Amazon S3](https://aws.amazon.com/public-datasets/goes/). I created a [web interface](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?domain=C&product=ABI-L2-MCMIP&hour=0) to easily download files from the Amazon archive. For scripted or bulk downloads, you should use `rclone` or `AWS CLI`. You may also download files from the [Environmental Data Commons](http://edc.occ-data.org/goes16/getdata/) and [NOAA CLASS](https://www.avl.class.noaa.gov/saa/products/search?sub_id=0&datatype_family=GRABIPRD&submit.x=25&submit.y=9).

This example uses the **level 2 _multiband_ formatted file for the _CONUS_ domain** ([ABI-L2-MCMIPC](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?domain=C&product=ABI-L2-MCMIP&hour=0)). The multiband file is easiest to work with becuase it contains all 16 channels on the same 2 km grid. However, some channels have higher resolution. Plotting the full resolution images will take some additional work, not described here. Specifically, you will have to download three separate files, one for each channel, and subsample the red channel 0.5 km grid to a 1 km grid.

I previously downloaded the following file from Amazon Web Services. The date and time this file corresponds to is during the [Pole Creek fire](https://inciweb.nwcg.gov/incident/6199/) in Utah on September 14. 2018. 

    OR_ABI-L2-MCMIPC-M3_G16_s20182570022128_e20182570024501_c20182570025006.nc

>OR     - Indicates the system is operational  
 ABI    - Instrument type  
 L2     - Level 2 Data  
 MCMIP  - Multichannel Cloud and Moisture Imagery products  
 c      - CONUS file (created every 5 minutes).  
 M3     - Scan mode  
 G16    - GOES-16  
 s##### - Scan start: 4 digit year, 3 digit day of year (Julian day), hour, minute, second, tenth second  
 e##### - Scan end
 c##### - File Creation  
 .nc    - NetCDF file extension  

More information about GOES-16 files can be found on [Amazon AWS](https://docs.opendata.aws/noaa-goes16/cics-readme.html).

## First, import the libraries we will use

In [1]:
%matplotlib inline

import numpy as np
from datetime import datetime, timedelta
from pyproj import Proj
import xarray
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

## Open the GOES-16 NetCDF File
Using xarray, I assign the opened file to the variable C for the CONUS domain.

In [2]:
FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20182570022128_e20182570024501_c20182570025006.nc'

C = xarray.open_dataset(FILE)

## Date and Time Information
Each file represents the data collected during one scan sequence for the domain. There are several different time stamps in this file, which are also found in the file's name.

In [3]:
# Scan's start time, converted to datetime object
scan_start = datetime.strptime(C.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')

# Scan's end time, converted to datetime object
scan_end = datetime.strptime(C.time_coverage_end, '%Y-%m-%dT%H:%M:%S.%fZ')

# File creation time, convert to datetime object
file_created = datetime.strptime(C.date_created, '%Y-%m-%dT%H:%M:%S.%fZ')

# The 't' variable is the scan's midpoint time
# I'm not a fan of numpy datetime, so I convert it to a regular datetime object
midpoint = str(C['t'].data)[:-8]
scan_mid = datetime.strptime(midpoint, '%Y-%m-%dT%H:%M:%S.%f')

print('Scan Start    : %s' % scan_start)
print('Scan midpoint : %s' % scan_mid)
print('Scan End      : %s' % scan_end)
print('File Created  : %s' % file_created)
print('Scan Duration : %.2f minutes' % ((scan_end-scan_start).seconds/60))

Scan Start    : 2018-09-14 00:22:12.800000
Scan midpoint : 2018-09-14 00:23:31.400000
Scan End      : 2018-09-14 00:24:50.100000
File Created  : 2018-09-14 00:25:00.600000
Scan Duration : 2.62 minutes


## True Color Recipe
Color images are a Red-Green-Blue (RGB) composite of three different channels. We will assign the following channels as our RGB values, according the the [GOES Quick Guide Reference](http://rammb.cira.colostate.edu/training/visit/quick_guides/Fire_Temperature_RGB.pdf):


|          -    | **RED**      | **GREEN**      | **BLUE**     |
|---------------|:------------:|:--------------:|:------------:|
| **Name**      | Shortwave Window | Cloud Partilce Size | Snow/Ice |
| **Wavelength**| 3.9 &#181;m | 2.2 &#181;m   | 1.6 &#181;m |
| **Channel**   |      7       |       6        |      5       |
| **Units**     | temperature range [273.15-333.15]<br>Gamma=1/.4 | reflectance range [0-1]  | reflectance range [0-75] |

The multiband formatted file we loaded is convenient becuase all the GOES channels are in the same NetCDF file. Next, we will assign our variables R, G, and B as the data for each channel.

In [4]:
# Confirm that each band is the wavelength we are interested in
for band in [7, 6, 5]:
    print("%s is %.2f %s" % (C['band_wavelength_C%02d' % band].long_name,
                             C['band_wavelength_C%02d' % band][0],
                             C['band_wavelength_C%02d' % band].units))

ABI band 7 central wavelength is 3.89 um
ABI band 6 central wavelength is 2.25 um
ABI band 5 central wavelength is 1.61 um
